# Preparing data
## I use pandas for easy manipulation

In [5]:
import pandas as pd
import numpy as np
import datetime

# Change DataFrame dtypes to match Database models
def model_for_db(df):
    df['program_year'] = pd.to_numeric(df['program_year'], errors='coerce')
    df['start_time'] = (df['start_date'].astype(str)+' '+df['start_time']).map(lambda x: datetime.datetime.strptime(x, '%Y%m%d %H:%M'))
    df['duration_in_seconds'] = pd.to_timedelta(df['duration_in_seconds'], unit='seconds')
    df['end_time'] = df['start_time']+df['duration_in_seconds']
    df = df.drop(['start_date', 'duration_in_seconds'], axis=1) 
    return df

# Load csv into Dataframe
filename = 'epg.csv'
df = pd.read_csv(filename)

# Fill n/a
df = df.fillna('n/a')

# Model for the db
df = model_for_db(df)

# What is the shape of the df
print("The data is {} cols by {} rows.\n".format(df.shape[1], df.shape[0]))
for column in df.columns:
    print(column, df[column].count())

print('\n'+'-'*31+'\n') 

print('The quantity of unique values in each col are the following:\n')
for column in df.columns:
    print(column, df[column].unique().size)
    
df.sample(3)

The data is 9 cols by 4897 rows.

channel_id 4897
channel_name 4897
channel_country 4897
start_time 4897
program_id 4897
program_original_title 4897
program_local_title 4897
program_year 2464
end_time 4897

-------------------------------

The quantity of unique values in each col are the following:

channel_id 3
channel_name 3
channel_country 3
start_time 4201
program_id 1879
program_original_title 358
program_local_title 372
program_year 44
end_time 4147


,channel_id,channel_name,channel_country,start_time,program_id,program_original_title,program_local_title,program_year,end_time
2510,69036687,RTE 1,IRL,2018-01-31 20:30:00,5741755326,Operation Transformation,Operation Transformation,2018.0,2018-01-31 21:00:00
3891,69036687,RTE 1,IRL,2018-02-17 05:20:00,12073476,Euronews,Euronews,NaN,2018-02-17 06:00:00
1213,69036687,RTE 1,IRL,2018-01-16 04:15:00,5698201877,Shortland Street,Shortland Street,2018.0,2018-01-16 04:40:00



<hr>

### *Observe there are cells in "program_original_title" and "program_year" that are null*


<hr>

# Add the df Showtimes to DB

In [6]:
import os
 
from django.db import IntegrityError
from tv import models

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

# Get or create Program with channel obj and row data
def create_program(row, channel):
    program, created = models.Program.objects.get_or_create(uid=row.program_id,channel=channel,
                                                           year=row.program_year,
                                                           local_title=row.program_local_title,
                                                           original_title=row.program_original_title
                                                           )
    if created:
        program.save()
        print("Program {} with id {} added to DB.".format(program.local_title, 
                                                                program.uid,
                                                               ))
    return program

def create_channel(row):
    channel, created = models.Channel.objects.get_or_create(uid=row.channel_id, 
                                                            name=row.channel_name, 
                                                            country_code=row.channel_country
                                                           )
    if created:
        channel.save()
        print('Channel {} from {} with id {} added to DB.'.format(channel.name, 
                                                                  channel.country_code, 
                                                                  channel.id
                                                                 ))
    return channel

def create_showtime(row,program):
    showtime, created = models.Showtime.objects.get_or_create(program=program, 
                                                              start_time=row.start_time, 
                                                              end_time=row.end_time
                                                             )
    if created:
        showtime.save()
        print('Showtime starting on {} added to {}.'.format(row.start_time, 
                                                            program
                                                           ))
    return showtime

              
# Create Showtimes from DataFrame           
def add_showtimes_from_dataframe(df):
    for index, row in df.iterrows():
        channel = create_channel(row)
        program = create_program(row, channel)
        create_showtime(row, program)
            
    print("Finished adding programs from {}".format(filename))
        



<hr>

# -v- Add programs to DB -v-

```python
addShowtimeFromDataFrame(df)
```

<hr>


In [10]:
add_showtimes_from_dataframe(df)

Channel HD1 (France) from FRA with id 44 added to DB.
Program À communiquer with id 461044 added to DB.
Showtime starting on 2018-01-01 00:00:00 added to Program: À communiquer with id 461044.
Channel La Sexta from ESP with id 45 added to DB.
Program Será anunciado with id 1887442 added to DB.
Showtime starting on 2018-01-01 00:00:00 added to Program: Será anunciado with id 1887442.
Channel RTE 1 from IRL with id 46 added to DB.
Program RTE 1 Programming with id 3452333967 added to DB.
Showtime starting on 2018-01-01 00:00:00 added to Program: RTE 1 Programming with id 3452333967.
Program L.A. Confidential with id 446381 added to DB.
Showtime starting on 2018-01-01 00:30:00 added to Program: L.A. Confidential with id 446381.
Program No controles with id 18063328 added to DB.
Showtime starting on 2018-01-01 00:45:00 added to Program: No controles with id 18063328.
Program Canal Ruleta with id 5072179538 added to DB.
Showtime starting on 2018-01-01 02:20:00 added to Program: Canal Ruleta

Program Grey's Anatomy with id 17875417 added to DB.
Showtime starting on 2018-01-01 17:20:00 added to Program: Grey's Anatomy with id 17875417.
Program Nuacht RTÉ with id 12766064 added to DB.
Showtime starting on 2018-01-01 17:45:00 added to Program: Nuacht RTÉ with id 12766064.
Showtime starting on 2018-01-01 17:45:00 added to Program: Cine with id 13929447.
Program The Angelus with id 12039424 added to DB.
Showtime starting on 2018-01-01 18:00:00 added to Program: The Angelus with id 12039424.
Program RTÉ News: Six One with id 11989490 added to DB.
Showtime starting on 2018-01-01 18:01:00 added to Program: RTÉ News: Six One with id 11989490.
Program Grey's Anatomy with id 17875411 added to DB.
Showtime starting on 2018-01-01 18:05:00 added to Program: Grey's Anatomy with id 17875411.
Program Senior Moments with id 4345613819 added to DB.
Showtime starting on 2018-01-01 18:30:00 added to Program: Senior Moments with id 4345613819.
Program laSexta Noticias 2ª Edición with id 31992322

Showtime starting on 2018-01-02 12:20:00 added to Program: Au nom de la vérité with id 5642731384.
Program Telly Bingo with id 12127388 added to DB.
Showtime starting on 2018-01-02 12:45:00 added to Program: Telly Bingo with id 12127388.
Showtime starting on 2018-01-02 13:00:00 added to Program: RTÉ News: One O'Clock with id 11939294.
Showtime starting on 2018-01-02 13:00:00 added to Program: Petits plats en équilibre with id 12044548.
Showtime starting on 2018-01-02 13:00:00 added to Program: laSexta Noticias 1ª Edición with id 11909047.
Showtime starting on 2018-01-02 13:05:00 added to Program: Urgences with id 12681871.
Showtime starting on 2018-01-02 13:20:00 added to Program: EastEnders with id 5612084711.
Showtime starting on 2018-01-02 13:50:00 added to Program: Urgences with id 12595264.
Showtime starting on 2018-01-02 13:55:00 added to Program: La Sexta Noticias: Jugones with id 4077761636.
Showtime starting on 2018-01-02 14:20:00 added to Program: Jugones: El análisis with id

Showtime starting on 2018-01-03 08:40:00 added to Program: Only Fools and Horses with id 887001.
Program Au nom de la vérité with id 5542565732 added to DB.
Showtime starting on 2018-01-03 08:40:00 added to Program: Au nom de la vérité with id 5542565732.
Program Crímenes imperfectos with id 17699664 added to DB.
Showtime starting on 2018-01-03 08:45:00 added to Program: Crímenes imperfectos with id 17699664.
Program Moguls & Movie Stars with id 5623744350 added to DB.
Showtime starting on 2018-01-03 09:10:00 added to Program: Moguls & Movie Stars with id 5623744350.
Program Au nom de la vérité with id 5617775796 added to DB.
Showtime starting on 2018-01-03 09:15:00 added to Program: Au nom de la vérité with id 5617775796.
Program Las primeras 48 horas with id 4804635930 added to DB.
Showtime starting on 2018-01-03 09:15:00 added to Program: Las primeras 48 horas with id 4804635930.
Program Au nom de la vérité with id 5547041839 added to DB.
Showtime starting on 2018-01-03 09:45:00 add

Showtime starting on 2018-01-04 00:00:00 added to Program: RTE 1 Programming with id 3452333967.
Showtime starting on 2018-01-04 00:00:00 added to Program: Será anunciado with id 1887442.
Program John Connors' America with id 4414546701 added to DB.
Showtime starting on 2018-01-04 00:55:00 added to Program: John Connors' America with id 4414546701.
Showtime starting on 2018-01-04 01:40:00 added to Program: Poker Caribbean Adventure with id 505649787.
Program Pet Island with id 3381498726 added to DB.
Showtime starting on 2018-01-04 01:55:00 added to Program: Pet Island with id 3381498726.
Showtime starting on 2018-01-04 02:10:00 added to Program: Comprando en casa with id 1076761458.
Program Frock Finders with id 5255597306 added to DB.
Showtime starting on 2018-01-04 02:25:00 added to Program: Frock Finders with id 5255597306.
Program Find Me a Home with id 5207549497 added to DB.
Showtime starting on 2018-01-04 02:50:00 added to Program: Find Me a Home with id 5207549497.
Showtime st

...

Showtime starting on 2018-02-24 19:00:00 added to Program: laSexta Noticias 2ª Edición with id 319923221.
Showtime starting on 2018-02-24 19:45:00 added to Program: La sexta meteo with id 768381956.
Showtime starting on 2018-02-24 19:55:00 added to Program: Petits plats en équilibre with id 12044548.
Program Training Day with id 5502206782 added to DB.
Showtime starting on 2018-02-24 20:00:00 added to Program: Training Day with id 5502206782.
Showtime starting on 2018-02-24 20:00:00 added to Program: La Sexta deportes with id 11922093.
Program Winning Streak with id 5837038340 added to DB.
Showtime starting on 2018-02-24 20:25:00 added to Program: Winning Streak with id 5837038340.
Program laSexta Noche with id 5827757542 added to DB.
Showtime starting on 2018-02-24 20:30:00 added to Program: laSexta Noche with id 5827757542.
Program Training Day with id 5516580252 added to DB.
Showtime starting on 2018-02-24 20:40:00 added to Program: Training Day with id 5516580252.
Showtime starting

Showtime starting on 2018-02-25 18:01:00 added to Program: RTÉ News: Six One with id 11989490.
Showtime starting on 2018-02-25 18:25:00 added to Program: Buques de guerra americanos with id 317111491.
Program Dancing with the Stars with id 5833196979 added to DB.
Showtime starting on 2018-02-25 18:30:00 added to Program: Dancing with the Stars with id 5833196979.
Showtime starting on 2018-02-25 19:00:00 added to Program: laSexta Noticias 2ª Edición with id 319923221.
Showtime starting on 2018-02-25 19:45:00 added to Program: La sexta meteo with id 768381956.
Program Ma part du gâteau with id 18425621 added to DB.
Showtime starting on 2018-02-25 20:00:00 added to Program: Ma part du gâteau with id 18425621.
Showtime starting on 2018-02-25 20:00:00 added to Program: La Sexta deportes with id 11922093.
Showtime starting on 2018-02-25 20:25:00 added to Program: Salvados with id 12114856.
Program Fair City with id 5837038263 added to DB.
Showtime starting on 2018-02-25 20:30:00 added to Pro

Showtime starting on 2018-02-26 16:15:00 added to Program: Más vale tarde with id 5835149689.
Showtime starting on 2018-02-26 16:20:00 added to Program: Grey's Anatomy with id 22271793.
Showtime starting on 2018-02-26 17:10:00 added to Program: Grey's Anatomy with id 19733321.
Showtime starting on 2018-02-26 17:40:00 added to Program: Nuacht RTÉ with id 12766064.
Showtime starting on 2018-02-26 18:00:00 added to Program: Grey's Anatomy with id 22301334.
Showtime starting on 2018-02-26 18:00:00 added to Program: The Angelus with id 12039424.
Showtime starting on 2018-02-26 18:01:00 added to Program: RTÉ News: Six One with id 11989490.
Program Grey's Anatomy with id 22301333 added to DB.
Showtime starting on 2018-02-26 18:55:00 added to Program: Grey's Anatomy with id 22301333.
Program Nationwide with id 5837038273 added to DB.
Showtime starting on 2018-02-26 19:00:00 added to Program: Nationwide with id 5837038273.
Showtime starting on 2018-02-26 19:00:00 added to Program: laSexta Notic

Showtime starting on 2018-02-27 13:55:00 added to Program: La Sexta Noticias: Jugones with id 4077761636.
Program Neighbours with id 5837068025 added to DB.
Showtime starting on 2018-02-27 14:00:00 added to Program: Neighbours with id 5837068025.
Showtime starting on 2018-02-27 14:20:00 added to Program: Jugones: El análisis with id 3286898281.
Showtime starting on 2018-02-27 14:25:00 added to Program: EastEnders with id 5837433990.
Showtime starting on 2018-02-27 14:30:00 added to Program: La sexta meteo with id 768381956.
Showtime starting on 2018-02-27 14:35:00 added to Program: Urgences with id 13135274.
Program Zapeando with id 5842310238 added to DB.
Showtime starting on 2018-02-27 14:45:00 added to Program: Zapeando with id 5842310238.
Program Grandma's Boy with id 1270938634 added to DB.
Showtime starting on 2018-02-27 15:00:00 added to Program: Grandma's Boy with id 1270938634.
Program Urgences with id 12603805 added to DB.
Showtime starting on 2018-02-27 15:25:00 added to Pro

Program Leaders' Questions with id 12140764 added to DB.
Showtime starting on 2018-02-28 11:50:00 added to Program: Leaders' Questions with id 12140764.
Program Au nom de la vérité with id 5827250859 added to DB.
Showtime starting on 2018-02-28 12:10:00 added to Program: Au nom de la vérité with id 5827250859.
Showtime starting on 2018-02-28 12:50:00 added to Program: Petits plats en équilibre with id 12044548.
Showtime starting on 2018-02-28 12:55:00 added to Program: Urgences with id 12860623.
Showtime starting on 2018-02-28 13:00:00 added to Program: laSexta Noticias 1ª Edición with id 11909047.
Showtime starting on 2018-02-28 13:00:00 added to Program: RTÉ News: One O'Clock with id 11939294.
Program Home And Away with id 5837038265 added to DB.
Showtime starting on 2018-02-28 13:30:00 added to Program: Home And Away with id 5837038265.
Showtime starting on 2018-02-28 13:40:00 added to Program: Urgences with id 13135274.
Showtime starting on 2018-02-28 13:55:00 added to Program: La 

# -^- Programs added to DB -^-

<hr>


# -v- Delete models -v-

In [9]:
import os

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

from tv import models
for object in models.Program.objects.all():
    object.delete()
for object in models.Channel.objects.all():
    object.delete()

# -^- Models deleted -^-